# Marabou's Split & Conquer Guide

This Notebook illustrates properties of the Split & Conquer (S&C) algorithm in Marabou, provides an overview of algorithm parameters, shows their impact on algorithm performance and offers some rules of thumb on how to choose parameter values for problems of unknown difficulty.

## About Split & Conquer algorithm
The algorithm is build on top of Marabou solver for neural network verification(NNV). Scalability is one of the chief hurdles for NNV. S&C algorithm is an effort-based algorithm, which dynamically decomposes queries which exceed the effort limit. 

Effort is inpractice specified in the form of timeout - initially *t_0*. Whenever a query exceeds its timeout, the S&C algorithm will split it into a number of new queries. The number of splits *n* is another parameter of the algorithm, and will result in *2^n* new queries created. This new generation of queries will be solved with an increased timeout, which is specified through the timeout factor *f*. Each of the queries will in turn be solved with the new timout and this process is repeated, until either:
1. a subquery is shown to be **satisfiable**
2. or **all** the generated sub-queries are **unsatisfiable**.

Here is an illustration of the S&C algorithm execution with 4 parallel workers:
![Execution of Split & Conquer algorithm](img/snc.png)

### How to split the query?
Queries can be split in various way - the two that have shown the most success are:

1. **input range splitting** for networks with few inputs and 
2. **early ReLU phase splitting** for perception networks.

Both approaches still require to choose which particular input or ReLU to split on. Marabou implements several heuristics for this choice. The crucial property for parallelization is that the resulting queries are of similar difficulty, which in terms of search space means that the splits are *balanced*. The metric to estimate balance we called **polarity** and is the default choice for S&C.

A crucial point is that the above splitting strategies generate queries whose disjucntion is equivalent to the original query. This is essential for completeness of the overall algorithm and trustworthiness of reported results.

## Getting a feel for S&C parameters

In this section we will run a series of examples to show how different parameter values affect execution.

### First, let's run a sequential problem

In [3]:
import sys

# Path to Marabou folder if you did not export it
# sys.path.append('/PATH/TO/Marabou')
sys.path.append('/home/aleks/git/NNVMarabou')

from maraboupy import Marabou

In [20]:
# Choose an input file that takes about 30seconds to solve 
# Network files are sorted in increasing order of difficulty
#nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_2_6.nnet"
#nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_2_5.nnet"
nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_5_2.nnet"

# Load the network from NNet file
net = Marabou.read_nnet(nnetFile)

# Encode property 3 - see properties/acas_property3.txt
lower_bounds = [-0.3035311561, -0.0095492966, 0.4933803236, 0.3,  0.3]
upper_bounds = [ -0.2985528119, 0.0095492966, 0.5, 0.5, 0.5]

for idx in range(len(net.inputVars[0])):
    net.setLowerBound(net.inputVars[0][idx], lower_bounds[idx])
    net.setUpperBound(net.inputVars[0][idx], upper_bounds[idx])

net.addInequality(net.outputVars[0], [+1.0,-1.0, 0.0, 0.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0,-1.0, 0.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0, 0.0,-1.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0, 0.0, 0.0,-1.0], 0.0)


In [17]:
%%time
defaultOpts = Marabou.createOptions()
vals1, stats1 = net.solve(options=defaultOpts)

unsat
Solving time: 824ms
CPU times: user 918 ms, sys: 0 ns, total: 918 ms
Wall time: 917 ms


In [18]:
%%time
sequentialSnC = Marabou.createOptions(snc=True)
vals1, stats1 = net.solve(options=sequentialSnC)

unsat
CPU times: user 1.2 s, sys: 0 ns, total: 1.2 s
Wall time: 1.27 s


In [21]:
%%time
SnC4workers = Marabou.createOptions(snc=True,numWorkers=4)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 17.3 s, sys: 71.4 ms, total: 17.4 s
Wall time: 10.6 s


In [26]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineDivides=2)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 17.4 s, sys: 31.8 ms, total: 17.4 s
Wall time: 10.6 s


In [27]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineDivides=2,
                                     initialDivides=2)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 11.1 s, sys: 11.9 ms, total: 11.1 s
Wall time: 4.54 s


In [28]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineDivides=2,
                                     initialDivides=2,
                                     initialTimeout=3)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 13 s, sys: 12 ms, total: 13 s
Wall time: 5.4 s


In [29]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineDivides=2,
                                     initialDivides=2,
                                     initialTimeout=1)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 11.9 s, sys: 11.9 ms, total: 11.9 s
Wall time: 3.79 s


In [30]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineDivides=2,
                                     initialDivides=2,
                                     initialTimeout=1,
                                     timeoutFactor=3)
vals1, stats1 = net.solve(options=SnC4workers)

unsat
CPU times: user 11.9 s, sys: 7.85 ms, total: 11.9 s
Wall time: 3.81 s
